# Faster R-CNN: Explained

## Utility Functions

In [ ]:
import torch
import torchvision
import cv2
import os
import json
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
from pytorch_grad_cam.ablation_layer import AblationLayerFasterRCNN
from pytorch_grad_cam import GradCAM, AblationCAM, EigenCAM, ScoreCAM, GradCAMPlusPlus, DeepFeatureFactorization
from pytorch_grad_cam.utils.model_targets import FasterRCNNBoxScoreTarget
from pytorch_grad_cam.utils.reshape_transforms import fasterrcnn_reshape_transform
from pytorch_grad_cam.utils.image import show_cam_on_image, scale_cam_image, show_factorization_on_image
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

# Defining COCO categories
coco_labels = ['__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
              'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'N/A', 
              'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 
              'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'N/A', 'backpack', 'umbrella',
              'N/A', 'N/A', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard',
              'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard',
              'surfboard', 'tennis racket', 'bottle', 'N/A', 'wine glass', 'cup', 'fork',
              'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange',
              'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
              'potted plant', 'bed', 'N/A', 'dining table', 'N/A', 'N/A', 'toilet',
              'N/A', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave',
              'oven', 'toaster', 'sink', 'refrigerator', 'N/A', 'book', 'clock', 'vase',
              'scissors', 'teddy bear', 'hair drier', 'toothbrush']
              
def read_image(dataset_dir, image_name) -> np.ndarray:
    """
    Read and preprocess an image from the dataset directory.

    Args:
        dataset_dir (str): The directory path of the dataset.
        image_name (str): The name of the image file.

    Returns:
        image (numpy.ndarray): The preprocessed image as a NumPy array.

    """
    # Read the image using OpenCV
    image = cv2.imread(dataset_dir + image_name)

    # Convert the image from BGR to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image


def predict(input_tensor, model, detection_threshold) -> tuple:
    """
    Perform object detection on an input tensor using a Faster R-CNN model.

    Args:
        input_tensor (torch.Tensor): The input tensor to perform object detection on.
        model (torch.nn.Module): The Faster R-CNN model.
        detection_threshold (float): The minimum confidence score threshold for object detection.

    Returns:
        boxes (numpy.ndarray): An array of bounding boxes for the detected objects.
        classes (list): A list of class labels for the detected objects.
        labels (numpy.ndarray): An array of class labels for the detected objects.
        indices (list): A list of indices corresponding to the detected objects.
    """
    # Perform object detection using the model
    outputs = model(input_tensor)

    # Extract the predicted classes, labels, scores, and bounding boxes
    pred_classes = [coco_labels[i] for i in outputs[0]['labels'].cpu().numpy()]
    pred_labels = outputs[0]['labels'].cpu().numpy()
    pred_scores = outputs[0]['scores'].detach().cpu().numpy()
    pred_bboxes = outputs[0]['boxes'].detach().cpu().numpy()

    # Initialize empty lists for storing the filtered results
    boxes, classes, labels, indices = [], [], [], []

    # Filter the results based on the detection threshold
    for index in range(len(pred_scores)):
        if pred_scores[index] >= detection_threshold:
            boxes.append(pred_bboxes[index].astype(np.int32))
            classes.append(pred_classes[index])
            labels.append(pred_labels[index])
            indices.append(index)

    # Convert the boxes to numpy array
    boxes = np.int32(boxes)

    return boxes, classes, labels, indices


def predict_with_scores(input_tensor, model, detection_threshold) -> tuple:
    """
    Perform object detection on an input tensor using a Faster R-CNN model and return the bounding boxes,
    class labels, labels, indices, and scores for the detected objects.

    Args:
        input_tensor (torch.Tensor): The input tensor to perform object detection on.
        model (torch.nn.Module): The Faster R-CNN model.
        detection_threshold (float): The minimum confidence score threshold for object detection.

    Returns:
        boxes (numpy.ndarray): An array of bounding boxes for the detected objects.
        classes (list): A list of class labels for the detected objects.
        labels (numpy.ndarray): An array of class labels for the detected objects.
        indices (list): A list of indices corresponding to the detected objects.
        scores (numpy.ndarray): An array of confidence scores for the detected objects.
    """
    # Perform object detection using the model
    outputs = model(input_tensor)

    # Extract the predicted classes, labels, scores, and bounding boxes
    pred_classes = [coco_labels[i] for i in outputs[0]['labels'].cpu().numpy()]
    pred_labels = outputs[0]['labels'].cpu().numpy()
    pred_scores = outputs[0]['scores'].detach().cpu().numpy()
    pred_bboxes = outputs[0]['boxes'].detach().cpu().numpy()

    # Initialize empty lists for storing the filtered results
    boxes, classes, labels, indices, scores = [], [], [], [], []

    # Filter the results based on the detection threshold
    for index in range(len(pred_scores)):
        if pred_scores[index] >= detection_threshold:
            boxes.append(pred_bboxes[index].astype(np.int32))
            classes.append(pred_classes[index])
            labels.append(pred_labels[index])
            indices.append(index)
            scores.append(pred_scores[index])

    # Convert the boxes to numpy array
    boxes = np.int32(boxes)

    return boxes, classes, labels, indices, scores


def draw_boxes(boxes, labels, classes, image) -> np.ndarray:
    """
    Draw bounding boxes and class labels on the input image.

    Args:
        boxes (numpy.ndarray): An array of bounding boxes for the detected objects.
        labels (numpy.ndarray): An array of class labels for the detected objects.
        classes (list): A list of class labels for the detected objects.
        image (numpy.ndarray): The input image.

    Returns:
        numpy.ndarray: The image with bounding boxes and class labels drawn on it.
    """
    for i, box in enumerate(boxes):
        cv2.rectangle(
            image,
            (int(box[0]), int(box[1])),
            (int(box[2]), int(box[3])),
            (0, 255, 0), 2
        )
        cv2.putText(image, classes[i], (int(box[0]), int(box[1] - 5)),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2,
                    lineType=cv2.LINE_AA)
    return image

def fasterrcnn_reshape_transform(x):
    """
    Reshape the output (activations) of a Faster R-CNN model to the shape of the input image.

    Args:
        x (dict): The output (activations) of a Faster R-CNN model.
        
    Returns:
        dict: The reshaped output (activations) of a Faster R-CNN model.
    """
    # Specify the target size (last two dimensions) of the reshaped output (height, width)
    target_size = x['pool'].size()[-2 : ]
    # Initialize an empty list for storing the activations
    activations = []
    # Iterate over the activations
    for key, value in x.items():
        # Resize the activation to the target size using bilinear interpolation
        activations.append(torch.nn.functional.interpolate(torch.abs(value), target_size, mode='bilinear'))
    # Concatenate the activations along the channel dimension
    activations = torch.cat(activations, axis=1)
    return activations

class FasterRCNNBoxScoreTarget:
    """ For every original detected bounding box specified in "bounding boxes",
        assign a score on how the current bounding boxes match it,
            1. In IOU
            2. In the classification score.
        If there is not a large enough overlap, or the category changed,
        assign a score of 0.

        The total score is the sum of all the box scores.
    """

    def __init__(self, labels, bounding_boxes, iou_threshold=0.5):
        self.labels = labels
        self.bounding_boxes = bounding_boxes
        self.iou_threshold = iou_threshold

    def __call__(self, model_outputs):
        output = torch.Tensor([0])

        if len(model_outputs["boxes"]) == 0:
            return output

        for box, label in zip(self.bounding_boxes, self.labels):
            box = torch.Tensor(box[None, :])

            ious = torchvision.ops.box_iou(box, model_outputs["boxes"])
            index = ious.argmax()
            if ious[0, index] > self.iou_threshold and model_outputs["labels"][index] == label:
                score = ious[0, index] + model_outputs["scores"][index]
                output = output + score
        return output
    
def renormalize_cam_in_bounding_boxes(boxes, image_float_np, grayscale_cam):
    """Normalize the CAM to be in the range [0, 1] 
    inside every bounding boxes, and zero outside of the bounding boxes. """
    renormalized_cam = np.zeros(grayscale_cam.shape, dtype=np.float32)
    images = []
    for x1, y1, x2, y2 in boxes:
        img = renormalized_cam * 0
        img[y1:y2, x1:x2] = scale_cam_image(grayscale_cam[y1:y2, x1:x2].copy())    
        images.append(img)
    
    renormalized_cam = np.max(np.float32(images), axis = 0)
    renormalized_cam = scale_cam_image(renormalized_cam)
    eigencam_image_renormalized = show_cam_on_image(image_float_np, renormalized_cam, use_rgb=True)
    image_with_bounding_boxes = draw_boxes(boxes, labels, classes, eigencam_image_renormalized)
    return image_with_bounding_boxes

## Inference & Evaluation (IoU & mAP)

The `torchvision.models.detection` module provides several pre-trained models for object detection, segmentation, and person keypoint detection, as well as some training utilities. For Faster R-CNN, the following models are available:

1. `fasterrcnn_resnet50_fpn`: This is a Faster R-CNN model with a ResNet-50 backbone and Feature Pyramid Network (FPN). It's pre-trained on the COCO train2017 dataset.

2. `fasterrcnn_mobilenet_v3_large_fpn`: This is a Faster R-CNN model with a MobileNetV3-Large backbone and FPN. It's also pre-trained on the COCO train2017 dataset.

3. `fasterrcnn_mobilenet_v3_large_320_fpn`: This is another Faster R-CNN model with a MobileNetV3-Large backbone and FPN, but designed for 320x320 input images. It's pre-trained on the COCO train2017 dataset.

### Inference

In [ ]:
# Defining constants
DATASET_DIR = "dataset/val2017/"
CONFIDENCE_THRESHOLD = 0.9
SAVE = False

# Loading the COCO dataset in "dataset/test2017" 
test_images = os.listdir(DATASET_DIR)

# Loading annotations
coco_gt = COCO('dataset/instances_val2017.json')

# Preparing the image info dictionary & filename to id dictionary
image_ids = coco_gt.getImgIds()
image_info = coco_gt.loadImgs(image_ids)
filename_to_id = {img['file_name']: img['id'] for img in image_info}

# Loading the pre-trained Faster R-CNN model trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=torchvision.models.detection.FasterRCNN_ResNet50_FPN_Weights.DEFAULT)
model.eval()

# Defining the transformation to be applied to images
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
split = "val"

# Creating the output directory
output_dir = "output (Faster_RCNN)/" + split + "/"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Initializing empty lists for storing the predictions
predictions = []

# Iterating over the images
for sample in test_images:

    # Reading the image
    img_id = filename_to_id[sample]
    image = np.array(Image.open(DATASET_DIR + sample).convert('RGB'))
    input_tensor = transform(image).unsqueeze(0)

    # Forward pass through the model
    with torch.no_grad():
        outputs = model(input_tensor)  

    # Parse outputs
    pred_boxes = outputs[0]['boxes'].data.cpu().numpy()
    pred_scores = outputs[0]['scores'].data.cpu().numpy()
    pred_labels = outputs[0]['labels'].data.cpu().numpy()

    # Adding predictions to the list
    for i, (box, score, label) in enumerate(zip(pred_boxes, pred_scores, pred_labels)):
        prediction = {
            'image_id': img_id,
            'category_id': int(label),
            'bbox': box.tolist(),
            'score': float(score)
        }
        predictions.append(prediction)

    # Drawing bounding boxes and class labels on the image and saving it iff SAVE is True
    if SAVE:
        boxes, classes, labels, indices = predict(input_tensor, model, CONFIDENCE_THRESHOLD)
        image = draw_boxes(boxes, labels, classes, image)
        Image.fromarray(image).save(output_dir + sample)

# Saving the predictions in a JSON file
with open(output_dir + 'predictions.json', 'w') as f:
    json.dump(predictions, f)

# Computing mAP
coco_dt = coco_gt.loadRes(output_dir + 'predictions.json')
coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
coco_eval.evaluate()
coco_eval.accumulate()
coco_eval.summarize()

mAP = coco_eval.stats[0]
print("mAP: ", mAP)

### Computing IoU

In [ ]:
# Load COCO annotations for the entire dataset
coco = COCO('dataset/instances_val2017.json')

# Load the pre-trained Faster R-CNN model trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights=torchvision.models.detection.FasterRCNN_ResNet50_FPN_Weights.DEFAULT)
model.eval()

# Initializing variables
total_iou = 0.0
total_boxes = 0

# Iterating over all images in the dataset
for img_id in coco.getImgIds():
    image = read_image(DATASET_DIR, coco.loadImgs(img_id)[0]['file_name'])
    input_tensor = transform(image).unsqueeze(0)

    # Getting ground truth boxes corresponding to the image id
    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)
    gt_boxes = []
    for ann in anns:
        gt_boxes.append(ann['bbox'])
    gt_boxes = np.array(gt_boxes)

    # Getting predicted boxes
    boxes, classes, labels, indices, scores = predict_with_scores(input_tensor, model, CONFIDENCE_THRESHOLD)
    keep_indices = [i for i, score in enumerate(scores) if score >= CONFIDENCE_THRESHOLD]

    # only keep boxes with score >= CONFIDENCE_THRESHOLD
    boxes = [boxes[i] for i in keep_indices]

    # Computing IoU
    iou = 0.0
    for gt_box in gt_boxes:
        # Find box with highest IoU
        max_iou = 0.0
        for box in boxes:
            iou = torchvision.ops.box_iou(torch.Tensor(box).unsqueeze(0), torch.Tensor(gt_box).unsqueeze(0))
            if iou > max_iou:
                max_iou = iou
        total_iou += max_iou
        total_boxes += 1

print("Average IoU: ", total_iou.item() / total_boxes)

## Activation Visualization

Activation visualization is a technique used in deep learning to understand and analyze the behavior of neural networks. It involves visualizing the activations or outputs of individual neurons or layers within a neural network when given a particular input.

The main purpose of activation visualization is to gain insights into how the network is processing and representing the input data. By visualizing the activations, we can identify which parts of the input are being emphasized or ignored by the network. This can help us understand the network's decision-making process and potentially identify any issues or biases in the model.

In general, the process involves the following steps:

1. Load the pre-trained model: Activation visualization is typically performed on pre-trained models. The first step is to load the model architecture and weights.

2. Prepare the input data: Depending on the task, you need to prepare the input data that the model expects. This could involve preprocessing, resizing, or normalizing the input images.

3. Forward pass: Pass the input data through the model to obtain the activations. This involves feeding the input data through the layers of the model and collecting the activations at the desired layer(s).

4. Visualize the activations: Once you have obtained the activations, you can visualize them using various techniques such as heatmaps, feature maps, or activation histograms. These visualizations can provide insights into the learned representations and patterns within the network.

In [ ]:
def activation_visualization(images, model, transform, show=False, save=True, split='val'):
    output_dir = "output/activation_visualization/" + split + "/"
    global first_layer_activations
    global last_layer_activations
    
    for sample in images:

        # Read the image from disk using the image_name
        image = read_image(DATASET_DIR, sample)
        image_tensor = transform(image).unsqueeze(0)

        # Forward pass through the model
        output = model(image_tensor)
        
        first_layer_activations = first_layer_activations.squeeze(0).detach().cpu().numpy()
        last_layer_activations = last_layer_activations.squeeze(0).detach().cpu().numpy()

        # Nomralize each of the activations
        for i in range(first_layer_activations.shape[0]):
            first_layer_activations[i] = (first_layer_activations[i] - first_layer_activations[i].min()) / (first_layer_activations[i].max() - first_layer_activations[i].min()) * 255

        for i in range(last_layer_activations.shape[0]):
            last_layer_activations[i] = (last_layer_activations[i] - last_layer_activations[i].min()) / (last_layer_activations[i].max() - last_layer_activations[i].min()) * 255
           
        # Visualize the activations
        # create a figure with three rows and three columns, first row is the original image (only one image)
        fig, axes = plt.subplots(3, 3, figsize=(10, 5))

        # Original image
        for i in range(0, 3):
            axes[0, i].axis('off')
        axes[0, 1].imshow(image)
        axes[0, 1].set_title("Original Image")

        # First layer activations for only the first 3 filters
        axes[1, 0].imshow(torchvision.transforms.ToPILImage()(first_layer_activations[0]), cmap="gray")
        axes[1, 0].set_title("First Layer Activations\nFilter 1")
        axes[1, 0].axis("off")

        axes[1, 1].imshow(torchvision.transforms.ToPILImage()(first_layer_activations[1]), cmap="gray")
        axes[1, 1].set_title("First Layer Activations\nFilter 2")
        axes[1, 1].axis("off")

        axes[1, 2].imshow(torchvision.transforms.ToPILImage()(first_layer_activations[2]), cmap="gray")
        axes[1, 2].set_title("First Layer Activations\nFilter 3")
        axes[1, 2].axis("off")

        # Last layer activations for only the first 3 filters
        axes[2, 0].imshow(torchvision.transforms.ToPILImage()(last_layer_activations[0]), cmap="gray")
        axes[2, 0].set_title("Last Layer Activations\nFilter 1")
        axes[2, 0].axis("off")

        axes[2, 1].imshow(torchvision.transforms.ToPILImage()(last_layer_activations[1]), cmap="gray")
        axes[2, 1].set_title("Last Layer Activations\nFilter 2")
        axes[2, 1].axis("off")

        axes[2, 2].imshow(torchvision.transforms.ToPILImage()(last_layer_activations[2]), cmap="gray")
        axes[2, 2].set_title("Last Layer Activations\nFilter 3")
        axes[2, 2].axis("off")
        plt.subplots_adjust(left=0.05, right=0.95, top=0.95, bottom=0.05, hspace=0.2, wspace=0.2)
        plt.tight_layout()

        if save:
            if not os.path.exists(output_dir):
                os.makedirs(output_dir)
            plt.savefig(output_dir + sample)

        if show:
            plt.show()

        first_layer_activations = None 
        last_layer_activations = None

# Constants
DATASET_DIR = "dataset/val2017/"
print("Loaded {} labels from COCO dataset".format(len(coco_labels)))

# Load the pre-trained Faster R-CNN model trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()

# Define the transformation to be applied to images
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

# Load the COCO dataset in "dataset/test2017"
test_images = os.listdir(DATASET_DIR)

# Specify the first and last layer of the model which will be used for visualization (hooked)
first_layer = model.backbone.body.relu
last_layer = model.backbone.body.layer4[2].conv3

# Initializing activations which will be used in the hook functions
first_layer_activations = None
last_layer_activations = None

# Defining hook for the first layer
def first_layer_hook(module, input, output):
    global first_layer_activations
    first_layer_activations = output

# Defining hook for the last layer
def last_layer_hook(module, input, output):
    global last_layer_activations
    last_layer_activations = output

# Registering the hooks
first_layer.register_forward_hook(first_layer_hook)
last_layer.register_forward_hook(last_layer_hook)

# Apply activation visualization
activation_visualization(test_images, model, transform, show=True, save=True, split='val')

## GradCAM

In [ ]:
DATASET_DIR = "dataset/val2017/"
test_images = os.listdir(DATASET_DIR)
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

Gradcam = GradCAM(model,
                    [model.backbone.body.layer4[2].conv3])

for image in test_images:
    image_name = image
    image = np.array(Image.open(DATASET_DIR + image))
    original_image = image.copy()
    image_float_np = np.float32(image) / 255
    
    input_tensor = transform(image_float_np)
    input_tensor = input_tensor.unsqueeze(0)

    # Run the model and display the detections
    boxes, classes, labels, indices = predict(input_tensor, model, 0.9)
    targets = [FasterRCNNBoxScoreTarget(labels=labels, bounding_boxes=boxes)]

    image_with_predictions = draw_boxes(boxes, labels, classes, image)

    # Computing GradCAM
    grayscale_gradcam = Gradcam(input_tensor=input_tensor, targets=targets)[0, :]
    gradcam = show_cam_on_image(image_float_np, grayscale_gradcam, use_rgb=True)
    if len(boxes) == 0:
        continue
    renormalized_gradcam = renormalize_cam_in_bounding_boxes(boxes, image_float_np, grayscale_gradcam)

    fig, axes = plt.subplots(1, 4, figsize=(10, 5))
    axes[0].imshow(original_image)
    axes[0].set_title("Input Image")
    axes[0].axis("off")

    # Image with predicted bounding boxes
    axes[1].imshow(image_with_predictions)
    axes[1].set_title("Predicted Boxes")
    axes[1].axis("off")

    # GradCAM heatmap
    axes[2].imshow(gradcam)
    axes[2].set_title("GradCAM Heatmap")
    axes[2].axis("off")

    # GradCAM heatmap renormalized in bounding boxes
    axes[3].imshow(renormalized_gradcam)
    axes[3].set_title("GradCAM Heatmap\nRenormalized in Bounding Boxes")
    axes[3].axis("off")


    plt.tight_layout()
    # Saving the images
    output_dir = "output/gradcam/val/"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    plt.savefig(output_dir + image_name)
    plt.show()

## GradCAM++

In [ ]:
DATASET_DIR = "dataset/val2017/"
test_images = os.listdir(DATASET_DIR)
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

Gradcampp = GradCAMPlusPlus(model,
                    [model.backbone.body.layer4[2].conv3])

for image in test_images:
    image_name = image
    image = np.array(Image.open(DATASET_DIR + image))
    original_image = image.copy()
    image_float_np = np.float32(image) / 255
    
    input_tensor = transform(image_float_np)
    input_tensor = input_tensor.unsqueeze(0)

    # Run the model and display the detections
    boxes, classes, labels, indices = predict(input_tensor, model, 0.9)
    if len(boxes) == 0:
        continue
    targets = [FasterRCNNBoxScoreTarget(labels=labels, bounding_boxes=boxes)]

    image_with_predictions = draw_boxes(boxes, labels, classes, image)

    # Computing GradCAM
    grayscale_gradcam = Gradcampp(input_tensor=input_tensor, targets=targets)[0, :]
    gradcam = show_cam_on_image(image_float_np, grayscale_gradcam, use_rgb=True)
    if len(boxes) == 0:
        continue
    renormalized_gradcam = renormalize_cam_in_bounding_boxes(boxes, image_float_np, grayscale_gradcam)

    fig, axes = plt.subplots(1, 4, figsize=(10, 5))
    axes[0].imshow(original_image)
    axes[0].set_title("Input Image")
    axes[0].axis("off")

    # Image with predicted bounding boxes
    axes[1].imshow(image_with_predictions)
    axes[1].set_title("Predicted Boxes")
    axes[1].axis("off")

    # GradCAM heatmap
    axes[2].imshow(gradcam)
    axes[2].set_title("GradCAM++ Heatmap")
    axes[2].axis("off")

    # GradCAM heatmap renormalized in bounding boxes
    axes[3].imshow(renormalized_gradcam)
    axes[3].set_title("GradCAM++ Heatmap\nRenormalized in Bounding Boxes")
    axes[3].axis("off")


    plt.tight_layout()
    # Saving the images
    output_dir = "output/gradcampp/val/"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    plt.savefig(output_dir + image_name)
    plt.show()

## EigenCAM

In [ ]:
DATASET_DIR = "dataset/val2017/"
test_images = os.listdir(DATASET_DIR)
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
target_layers = [model.backbone]

Eigencam = EigenCAM(model,
               target_layers, 
               reshape_transform=fasterrcnn_reshape_transform)

for image in test_images:
    image_name = image
    image = np.array(Image.open(DATASET_DIR + image))
    original_image = image.copy()
    image_float_np = np.float32(image) / 255
    
    input_tensor = transform(image_float_np)
    input_tensor = input_tensor.unsqueeze(0)

    # Run the model and display the detections
    boxes, classes, labels, indices = predict(input_tensor, model, 0.9)
    targets = [FasterRCNNBoxScoreTarget(labels=labels, bounding_boxes=boxes)]

    image_with_predictions = draw_boxes(boxes, labels, classes, image)

    # Computing EigenCAM
    grayscale_eigencam = Eigencam(input_tensor=input_tensor, targets=targets)[0, :]
    eigencam_image = show_cam_on_image(image_float_np, grayscale_eigencam, use_rgb=True)
    if len(boxes) == 0:
        continue
    renormalized_eigencam_image = renormalize_cam_in_bounding_boxes(boxes, image_float_np, grayscale_eigencam)

    fig, axes = plt.subplots(1, 4, figsize=(10, 5))
    axes[0].imshow(original_image)
    axes[0].set_title("Input Image")
    axes[0].axis("off")

    # Image with predicted bounding boxes
    axes[1].imshow(image_with_predictions)
    axes[1].set_title("Predicted Boxes")
    axes[1].axis("off")

    # EigenCAM heatmap
    axes[2].imshow(eigencam_image)
    axes[2].set_title("EigenCAM Heatmap")
    axes[2].axis("off")

    # EigenCAM heatmap renormalized in bounding boxes
    axes[3].imshow(renormalized_eigencam_image)
    axes[3].set_title("EigenCAM Heatmap\nRenormalized in Bounding Boxes")
    axes[3].axis("off")


    plt.tight_layout()
    # Saving the images
    output_dir = "output/eigencam/val/"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    plt.savefig(output_dir + image_name)
    plt.show()

## AblationCAM

In [ ]:
DATASET_DIR = "dataset/val2017/"
test_images = os.listdir(DATASET_DIR)
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
target_layers = [model.backbone]

Ablationcam = AblationCAM(model,
                  target_layers, 
                  reshape_transform=fasterrcnn_reshape_transform,
                  ablation_layer=AblationLayerFasterRCNN(),
                  ratio_channels_to_ablate=0.01)

for image in test_images:
    image_name = image
    image = np.array(Image.open(DATASET_DIR + image))
    original_image = image.copy()
    image_float_np = np.float32(image) / 255
    
    input_tensor = transform(image_float_np)
    input_tensor = input_tensor.unsqueeze(0)

    # Run the model and display the detections
    boxes, classes, labels, indices = predict(input_tensor, model, 0.9)
    targets = [FasterRCNNBoxScoreTarget(labels=labels, bounding_boxes=boxes)]

    image_with_predictions = draw_boxes(boxes, labels, classes, image)

    # Computing AblationCAM
    grayscale_ablationcam = Ablationcam(input_tensor=input_tensor, targets=targets)[0, :]
    ablationcam = show_cam_on_image(image_float_np, grayscale_ablationcam, use_rgb=True)
    if len(boxes) == 0:
        continue
    renormalized_ablationcam = renormalize_cam_in_bounding_boxes(boxes, image_float_np, grayscale_ablationcam)

    fig, axes = plt.subplots(1, 4, figsize=(10, 5))
    axes[0].imshow(original_image)
    axes[0].set_title("Input Image")
    axes[0].axis("off")

    # Image with predicted bounding boxes
    axes[1].imshow(image_with_predictions)
    axes[1].set_title("Predicted Boxes")
    axes[1].axis("off")

    # AblationCAM heatmap
    axes[2].imshow(ablationcam)
    axes[2].set_title("AblationCAM Heatmap")
    axes[2].axis("off")

    # AblationCAM heatmap renormalized in bounding boxes
    axes[3].imshow(renormalized_ablationcam)
    axes[3].set_title("AblationCAM Heatmap\nRenormalized in Bounding Boxes")
    axes[3].axis("off")


    plt.tight_layout()
    # Saving the images
    output_dir = "output/ablation/val/"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    plt.savefig(output_dir + image_name)
    plt.show()

## Deep Feature Factorizations.

print(model.roi_heads.box_roi_pool.output_size[0])

In [ ]:
DATASET_DIR = "dataset/val2017/"
test_images = os.listdir(DATASET_DIR)
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

dff = DeepFeatureFactorization(model=model, target_layer=model.backbone.body.layer3[5].conv3, computation_on_concepts=model.roi_heads.box_predictor.cls_score)


for image in test_images:
    image_name = image
    image = np.array(Image.open(DATASET_DIR + image))
    original_image = image.copy()
    image_float_np = np.float32(image) / 255
    
    input_tensor = transform(image_float_np)
    input_tensor = input_tensor.unsqueeze(0)

    # Run the model and display the detections
    boxes, classes, labels, indices = predict(input_tensor, model, 0.9)

    image_with_predictions = draw_boxes(boxes, labels, classes, image)
    
    # Computing Deep Feature Factorization
    concepts_2, batch_explanations_2, concept_scores_2 = dff(input_tensor, n_components = 2)
    visualization_2 = show_factorization_on_image(image_float_np, 
                                                batch_explanations_2[0],
                                                image_weight=0.3)
    
    # Computing again for n_components = 3
    concepts_3, batch_explanations_3, concept_scores_3 = dff(input_tensor, n_components = 3)
    visualization_3 = show_factorization_on_image(image_float_np, 
                                                batch_explanations_3[0],
                                                image_weight=0.3)
    
    # Computing again for n_components = 5
    concepts_5, batch_explanations_5, concept_scores_5 = dff(input_tensor, n_components = 5)
    visualization_5 = show_factorization_on_image(image_float_np, 
                                                batch_explanations_5[0],
                                                image_weight=0.3)
        
    if len(boxes) == 0:
        continue

    fig, axes = plt.subplots(1, 5, figsize=(10, 5))
    axes[0].imshow(original_image)
    axes[0].set_title("Input Image")
    axes[0].axis("off")
    
    # Image with predicted bounding boxes
    axes[1].imshow(image_with_predictions)
    axes[1].set_title("Predicted Boxes")
    axes[1].axis("off")

    # Deep Feature Factorization heatmap
    axes[2].imshow(visualization_2)
    axes[2].set_title("DFF Heatmap (2)")
    axes[2].axis("off")

    # Deep Feature Factorization heatmap
    axes[3].imshow(visualization_3)
    axes[3].set_title("DFF Heatmap (3)")
    axes[3].axis("off")

    # Deep Feature Factorization heatmap
    axes[4].imshow(visualization_5)
    axes[4].set_title("DFF Heatmap (5)")
    axes[4].axis("off")

    plt.tight_layout()
    # Saving the images
    output_dir = "output/dff/val/"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    plt.savefig(output_dir + image_name)
    plt.show()

## ScoreCAM

- So far it causes the computer to crash. 

In [ ]:
DATASET_DIR = "dataset/val2017/"
test_images = os.listdir(DATASET_DIR)
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval()
transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])
target_layers = [model.backbone.body.layer4[2].conv3]

Scorecam = ScoreCAM(model,
                target_layers)

for image in test_images:
    image_name = image
    image = np.array(Image.open(DATASET_DIR + image))
    original_image = image.copy()
    image_float_np = np.float32(image) / 255
    
    input_tensor = transform(image_float_np)
    input_tensor = input_tensor.unsqueeze(0)

    # Run the model and display the detections
    boxes, classes, labels, indices = predict(input_tensor, model, 0.9)
    targets = [FasterRCNNBoxScoreTarget(labels=labels, bounding_boxes=boxes)]

    image_with_predictions = draw_boxes(boxes, labels, classes, image)

    # Computing ScoreCAM
    grayscale_scorecam = Scorecam(input_tensor=input_tensor, targets=targets)[0, :]
    scorecam = show_cam_on_image(image_float_np, grayscale_scorecam, use_rgb=True)
    if len(boxes) == 0:
        continue
    renormalized_scorecam = renormalize_cam_in_bounding_boxes(boxes, image_float_np, grayscale_scorecam)

    fig, axes = plt.subplots(1, 4, figsize=(10, 5))
    axes[0].imshow(original_image)
    axes[0].set_title("Input Image")
    axes[0].axis("off")

    # Image with predicted bounding boxes
    axes[1].imshow(image_with_predictions)
    axes[1].set_title("Predicted Boxes")
    axes[1].axis("off")

    # ScoreCAM heatmap
    axes[2].imshow(scorecam)
    axes[2].set_title("ScoreCAM Heatmap")
    axes[2].axis("off")

    # ScoreCAM heatmap renormalized in bounding boxes
    axes[3].imshow(renormalized_scorecam)
    axes[3].set_title("ScoreCAM Heatmap\nRenormalized in Bounding Boxes")
    axes[3].axis("off")

    plt.tight_layout()
    # Saving the images
    output_dir = "output/scorecam/val/"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    plt.savefig(output_dir + image_name)
    plt.show()